dataset_fl

In [1]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random
import sys


# -------------------- Device Setup --------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------- 데이터 로드 및 전처리 --------------------
try:
    df = pd.read_json('dataset_fl.json', lines=True)
    print("✅ JSON 파일 로드 성공.")
except Exception as e:
    print(f"❌ Error: JSON 파일 로드 실패. (오류: {e})")
    sys.exit()

df = df[['user_id', 'business_id', 'review_stars', 'text']].dropna()

# -------------------- SBERT 모델 로딩 (한 번만) --------------------
print("SBERT 모델 로딩 중...")
sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print("SBERT 모델 로딩 완료.")
context_dim = sbert_model.get_sentence_embedding_dimension()

# SBERT 문맥 벡터 생성 (전체 데이터에 대해 한 번만)
print("SBERT 문맥 벡터 생성 중...")
all_context_vectors = sbert_model.encode(df['text'].tolist(), show_progress_bar=True)
df['context_vectors'] = list(all_context_vectors)

# -------------------- Dataset 클래스 정의 --------------------
class UCAMDataset(Dataset):
    def __init__(self, users, items, ratings, contexts):
        self.users = users
        self.items = items
        self.ratings = ratings
        self.contexts = contexts

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.users[idx], dtype=torch.long),
            torch.tensor(self.items[idx], dtype=torch.long),
            torch.tensor(self.contexts[idx], dtype=torch.float32),
            torch.tensor(self.ratings[idx], dtype=torch.float32)
        )

# -------------------- 모델 정의 --------------------
class UCAM(nn.Module):
    def __init__(self, num_users, num_items, context_dim, embed_dim, hidden_dims):
        super().__init__()
        self.user_embed = nn.Embedding(num_users + 1, embed_dim)
        self.item_embed = nn.Embedding(num_items + 1, embed_dim)
        
        layers = []
        input_dim = embed_dim * 2 + context_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, h_dim))
            layers.append(nn.ReLU())
            input_dim = h_dim
        
        layers.append(nn.Linear(input_dim, 1))
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, user_ids, item_ids, context_vecs):
        u = self.user_embed(user_ids)
        i = self.item_embed(item_ids)
        x = torch.cat([u, i, context_vecs], dim=-1)
        return self.fc_layers(x).squeeze()

# -------------------- 평가 지표 함수 --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for users, items, contexts, ratings in data_loader:
            users, items, contexts, ratings = users.to(device), items.to(device), contexts.to(device), ratings.to(device)
            output = model(users, items, contexts)
            # TypeError 해결: output을 1차원 배열로 변환
            preds.extend(output.cpu().numpy().reshape(-1))
            targets.extend(ratings.cpu().numpy().reshape(-1))

    preds, targets = np.array(preds), np.array(targets)
    mae = mean_absolute_error(targets, preds)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    mape = mean_absolute_percentage_error(targets, preds)
    return mae, rmse, mape

# -------------------- 학습 및 평가 함수 (단일 실행) --------------------
def train_and_evaluate_run(params, train_df, val_df, test_df, run_num, is_search=False):
    # Map user/item IDs to indices for the current run's data
    user2idx = {uid: i for i, uid in enumerate(train_df['user_id'].unique())}
    item2idx = {iid: i for i, iid in enumerate(train_df['business_id'].unique())}
    unknown_user_idx = len(user2idx)
    unknown_item_idx = len(item2idx)
    
    train_df['user_idx'] = train_df['user_id'].map(user2idx)
    train_df['item_idx'] = train_df['business_id'].map(item2idx)
    val_df['user_idx'] = val_df['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
    val_df['item_idx'] = val_df['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)
    test_df['user_idx'] = test_df['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
    test_df['item_idx'] = test_df['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)

    train_dataset = UCAMDataset(train_df['user_idx'].values, train_df['item_idx'].values, train_df['review_stars'].values, np.stack(train_df['context_vectors'].values))
    val_dataset = UCAMDataset(val_df['user_idx'].values, val_df['item_idx'].values, val_df['review_stars'].values, np.stack(val_df['context_vectors'].values))
    test_dataset = UCAMDataset(test_df['user_idx'].values, test_df['item_idx'].values, test_df['review_stars'].values, np.stack(test_df['context_vectors'].values))

    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)
    
    model = UCAM(num_users=len(user2idx), num_items=len(item2idx), context_dim=context_dim,
                 embed_dim=params['embed_dim'], hidden_dims=params['hidden_dims']).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    criterion = nn.MSELoss()
    model_path = f'temp_ucam_model_run_{run_num}.pt'

    best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = params['patience']
    min_delta = 1e-4
    epochs = 50

    for epoch in range(epochs):
        model.train()
        for user_ids, item_ids, context_vectors, stars in train_loader:
            user_ids, item_ids, context_vectors, stars = user_ids.to(device), item_ids.to(device), context_vectors.to(device), stars.to(device)
            optimizer.zero_grad()
            predictions = model(user_ids, item_ids, context_vectors)
            loss = criterion(predictions, stars)
            loss.backward()
            optimizer.step()
        
        val_mae, val_rmse, val_mape = evaluate_model(model, val_loader, device)

        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                if is_search:
                    break
                else:
                    print(f"조기 종료 발생. (실행 {run_num})")
                    break
    
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
    
    test_mae, test_rmse, test_mape = evaluate_model(model, test_loader, device)
    
    if os.path.exists(model_path):
        os.remove(model_path)
    
    return test_mae, test_rmse, test_mape, best_val_rmse

# -------------------- 메인 실행 루틴 --------------------
def main():
    # -------------------- 1. 하이퍼파라미터 랜덤 서치 단계 --------------------
    param_grid = {
        'embed_dim': [32, 64, 128],
        'hidden_dims': [[64, 32], [128, 64], [256, 128]],
        'learning_rate': [0.0005, 0.001, 0.002],
        'batch_size': [128, 256, 512],
        'patience': [5, 7, 10]
    }
    num_trials = 10
    best_params = None
    best_val_rmse_search = float('inf')
    
    print(f"\n--- 1단계: 하이퍼파라미터 랜덤 서치 ({num_trials}회) 시작 ---")
    
    # 랜덤 서치를 위한 고정된 데이터 분할
    train_val_df_search, test_df_final = train_test_split(df, test_size=0.2, random_state=42)
    train_df_search, val_df_search = train_test_split(train_val_df_search, test_size=1/8, random_state=42)
    
    for i in range(num_trials):
        current_params = {k: random.choice(v) for k, v in param_grid.items()}
        print(f"\n--- 시도 {i+1}/{num_trials} --- 파라미터: {current_params}")
        
        # 최적의 파라미터는 검증 RMSE를 기준으로 찾습니다. (is_search=True)
        _, _, _, current_val_rmse = train_and_evaluate_run(current_params, train_df_search.copy(), val_df_search.copy(), test_df_final.copy(), f'search_{i+1}', is_search=True)
        
        print(f"    --> 시도 {i+1} 검증 RMSE: {current_val_rmse:.4f}")
        
        if current_val_rmse < best_val_rmse_search:
            best_val_rmse_search = current_val_rmse
            best_params = current_params
            print(f"    --> 새로운 최적 RMSE 발견: {best_val_rmse_search:.4f} (파라미터: {best_params})")

    print(f"\n✅ 1단계 완료. 최적 파라미터: {best_params}")

    # -------------------- 2. 최적 파라미터로 5회 반복 최종 평가 단계 --------------------
    if best_params:
        all_rmse, all_mae, all_mape = [], [], []
        num_runs = 5
        print(f"\n--- 2단계: 최적 파라미터로 {num_runs}회 반복 평가 시작 ---")

        for i in range(num_runs):
            # 매번 새로운 무작위 데이터 분할 사용 (random_state=42, 43, ...)
            random_state = 42 + i
            train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=random_state)
            train_df, val_df = train_test_split(train_val_df, test_size=1/8, random_state=random_state)

            print(f"\n--- 최종 실행 {i+1}/{num_runs} 시작 (Random State: {random_state}) ---")
            
            # 최종 테스트 성능을 측정합니다. (is_search=False)
            test_mae, test_rmse, test_mape, _ = train_and_evaluate_run(best_params, train_df.copy(), val_df.copy(), test_df.copy(), f'final_{i+1}', is_search=False)
            
            print(f"\n✅ [UCAM] {i+1}번째 테스트 결과: RMSE={test_rmse:.4f}, MAE={test_mae:.4f}, MAPE={test_mape:.2f}%")
            
            all_rmse.append(test_rmse)
            all_mae.append(test_mae)
            all_mape.append(test_mape)

        # 최종 평균 계산 및 출력
        avg_rmse = np.mean(all_rmse)
        std_rmse = np.std(all_rmse)
        avg_mae = np.mean(all_mae)
        std_mae = np.std(all_mae)
        avg_mape = np.mean(all_mape)
        std_mape = np.std(all_mape)

        print("\n\n==================== 최종 5회 반복 평균 결과 ====================")
        print(f" - 평균 RMSE : {avg_rmse:.4f} +/- {std_rmse:.4f}")
        print(f" - 평균 MAE  : {avg_mae:.4f} +/- {std_mae:.4f}")

if __name__ == "__main__":
    main()

C:\Users\alche\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Using device: cuda
✅ JSON 파일 로드 성공.
SBERT 모델 로딩 중...
SBERT 모델 로딩 완료.
SBERT 문맥 벡터 생성 중...


Batches: 100%|██████████| 15653/15653 [05:25<00:00, 48.11it/s] 



--- 1단계: 하이퍼파라미터 랜덤 서치 (10회) 시작 ---

--- 시도 1/10 --- 파라미터: {'embed_dim': 32, 'hidden_dims': [128, 64], 'learning_rate': 0.002, 'batch_size': 128, 'patience': 5}
    --> 시도 1 검증 RMSE: 0.7588
    --> 새로운 최적 RMSE 발견: 0.7588 (파라미터: {'embed_dim': 32, 'hidden_dims': [128, 64], 'learning_rate': 0.002, 'batch_size': 128, 'patience': 5})

--- 시도 2/10 --- 파라미터: {'embed_dim': 32, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 128, 'patience': 7}
    --> 시도 2 검증 RMSE: 0.7539
    --> 새로운 최적 RMSE 발견: 0.7539 (파라미터: {'embed_dim': 32, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 128, 'patience': 7})

--- 시도 3/10 --- 파라미터: {'embed_dim': 128, 'hidden_dims': [128, 64], 'learning_rate': 0.002, 'batch_size': 256, 'patience': 5}
    --> 시도 3 검증 RMSE: 0.8111

--- 시도 4/10 --- 파라미터: {'embed_dim': 64, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 512, 'patience': 5}
    --> 시도 4 검증 RMSE: 0.7754

--- 시도 5/10 --- 파라미터: {'embed_dim': 64, 'hidden_dims': [128, 64], 

dataset_la

In [3]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random
import sys


# -------------------- Device Setup --------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------- 데이터 로드 및 전처리 --------------------
try:
    df = pd.read_json('dataset_la.json', lines=True)
    print("✅ JSON 파일 로드 성공.")
except Exception as e:
    print(f"❌ Error: JSON 파일 로드 실패. (오류: {e})")
    sys.exit()

df = df[['user_id', 'business_id', 'review_stars', 'text']].dropna()

# -------------------- SBERT 모델 로딩 (한 번만) --------------------
print("SBERT 모델 로딩 중...")
sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print("SBERT 모델 로딩 완료.")
context_dim = sbert_model.get_sentence_embedding_dimension()

# SBERT 문맥 벡터 생성 (전체 데이터에 대해 한 번만)
print("SBERT 문맥 벡터 생성 중...")
all_context_vectors = sbert_model.encode(df['text'].tolist(), show_progress_bar=True)
df['context_vectors'] = list(all_context_vectors)

# -------------------- Dataset 클래스 정의 --------------------
class UCAMDataset(Dataset):
    def __init__(self, users, items, ratings, contexts):
        self.users = users
        self.items = items
        self.ratings = ratings
        self.contexts = contexts

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.users[idx], dtype=torch.long),
            torch.tensor(self.items[idx], dtype=torch.long),
            torch.tensor(self.contexts[idx], dtype=torch.float32),
            torch.tensor(self.ratings[idx], dtype=torch.float32)
        )

# -------------------- 모델 정의 --------------------
class UCAM(nn.Module):
    def __init__(self, num_users, num_items, context_dim, embed_dim, hidden_dims):
        super().__init__()
        self.user_embed = nn.Embedding(num_users + 1, embed_dim)
        self.item_embed = nn.Embedding(num_items + 1, embed_dim)
        
        layers = []
        input_dim = embed_dim * 2 + context_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, h_dim))
            layers.append(nn.ReLU())
            input_dim = h_dim
        
        layers.append(nn.Linear(input_dim, 1))
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, user_ids, item_ids, context_vecs):
        u = self.user_embed(user_ids)
        i = self.item_embed(item_ids)
        x = torch.cat([u, i, context_vecs], dim=-1)
        return self.fc_layers(x).squeeze()

# -------------------- 평가 지표 함수 --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for users, items, contexts, ratings in data_loader:
            users, items, contexts, ratings = users.to(device), items.to(device), contexts.to(device), ratings.to(device)
            output = model(users, items, contexts)
            # TypeError 해결: output을 1차원 배열로 변환
            preds.extend(output.cpu().numpy().reshape(-1))
            targets.extend(ratings.cpu().numpy().reshape(-1))

    preds, targets = np.array(preds), np.array(targets)
    mae = mean_absolute_error(targets, preds)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    mape = mean_absolute_percentage_error(targets, preds)
    return mae, rmse, mape

# -------------------- 학습 및 평가 함수 (단일 실행) --------------------
def train_and_evaluate_run(params, train_df, val_df, test_df, run_num, is_search=False):
    # Map user/item IDs to indices for the current run's data
    user2idx = {uid: i for i, uid in enumerate(train_df['user_id'].unique())}
    item2idx = {iid: i for i, iid in enumerate(train_df['business_id'].unique())}
    unknown_user_idx = len(user2idx)
    unknown_item_idx = len(item2idx)
    
    train_df['user_idx'] = train_df['user_id'].map(user2idx)
    train_df['item_idx'] = train_df['business_id'].map(item2idx)
    val_df['user_idx'] = val_df['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
    val_df['item_idx'] = val_df['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)
    test_df['user_idx'] = test_df['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
    test_df['item_idx'] = test_df['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)

    train_dataset = UCAMDataset(train_df['user_idx'].values, train_df['item_idx'].values, train_df['review_stars'].values, np.stack(train_df['context_vectors'].values))
    val_dataset = UCAMDataset(val_df['user_idx'].values, val_df['item_idx'].values, val_df['review_stars'].values, np.stack(val_df['context_vectors'].values))
    test_dataset = UCAMDataset(test_df['user_idx'].values, test_df['item_idx'].values, test_df['review_stars'].values, np.stack(test_df['context_vectors'].values))

    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)
    
    model = UCAM(num_users=len(user2idx), num_items=len(item2idx), context_dim=context_dim,
                 embed_dim=params['embed_dim'], hidden_dims=params['hidden_dims']).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    criterion = nn.MSELoss()
    model_path = f'temp_ucam_model_run_{run_num}.pt'

    best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = params['patience']
    min_delta = 1e-4
    epochs = 50

    for epoch in range(epochs):
        model.train()
        for user_ids, item_ids, context_vectors, stars in train_loader:
            user_ids, item_ids, context_vectors, stars = user_ids.to(device), item_ids.to(device), context_vectors.to(device), stars.to(device)
            optimizer.zero_grad()
            predictions = model(user_ids, item_ids, context_vectors)
            loss = criterion(predictions, stars)
            loss.backward()
            optimizer.step()
        
        val_mae, val_rmse, val_mape = evaluate_model(model, val_loader, device)

        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                if is_search:
                    break
                else:
                    print(f"조기 종료 발생. (실행 {run_num})")
                    break
    
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
    
    test_mae, test_rmse, test_mape = evaluate_model(model, test_loader, device)
    
    if os.path.exists(model_path):
        os.remove(model_path)
    
    return test_mae, test_rmse, test_mape, best_val_rmse

# -------------------- 메인 실행 루틴 --------------------
def main():
    # -------------------- 1. 하이퍼파라미터 랜덤 서치 단계 --------------------
    param_grid = {
        'embed_dim': [32, 64, 128],
        'hidden_dims': [[64, 32], [128, 64], [256, 128]],
        'learning_rate': [0.0005, 0.001, 0.002],
        'batch_size': [128, 256, 512],
        'patience': [5, 7, 10]
    }
    num_trials = 10
    best_params = None
    best_val_rmse_search = float('inf')
    
    print(f"\n--- 1단계: 하이퍼파라미터 랜덤 서치 ({num_trials}회) 시작 ---")
    
    # 랜덤 서치를 위한 고정된 데이터 분할
    train_val_df_search, test_df_final = train_test_split(df, test_size=0.2, random_state=42)
    train_df_search, val_df_search = train_test_split(train_val_df_search, test_size=1/8, random_state=42)
    
    for i in range(num_trials):
        current_params = {k: random.choice(v) for k, v in param_grid.items()}
        print(f"\n--- 시도 {i+1}/{num_trials} --- 파라미터: {current_params}")
        
        # 최적의 파라미터는 검증 RMSE를 기준으로 찾습니다. (is_search=True)
        _, _, _, current_val_rmse = train_and_evaluate_run(current_params, train_df_search.copy(), val_df_search.copy(), test_df_final.copy(), f'search_{i+1}', is_search=True)
        
        print(f"    --> 시도 {i+1} 검증 RMSE: {current_val_rmse:.4f}")
        
        if current_val_rmse < best_val_rmse_search:
            best_val_rmse_search = current_val_rmse
            best_params = current_params
            print(f"    --> 새로운 최적 RMSE 발견: {best_val_rmse_search:.4f} (파라미터: {best_params})")

    print(f"\n✅ 1단계 완료. 최적 파라미터: {best_params}")

    # -------------------- 2. 최적 파라미터로 5회 반복 최종 평가 단계 --------------------
    if best_params:
        all_rmse, all_mae, all_mape = [], [], []
        num_runs = 5
        print(f"\n--- 2단계: 최적 파라미터로 {num_runs}회 반복 평가 시작 ---")

        for i in range(num_runs):
            # 매번 새로운 무작위 데이터 분할 사용 (random_state=42, 43, ...)
            random_state = 42 + i
            train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=random_state)
            train_df, val_df = train_test_split(train_val_df, test_size=1/8, random_state=random_state)

            print(f"\n--- 최종 실행 {i+1}/{num_runs} 시작 (Random State: {random_state}) ---")
            
            # 최종 테스트 성능을 측정합니다. (is_search=False)
            test_mae, test_rmse, test_mape, _ = train_and_evaluate_run(best_params, train_df.copy(), val_df.copy(), test_df.copy(), f'final_{i+1}', is_search=False)
            
            print(f"\n✅ [UCAM] {i+1}번째 테스트 결과: RMSE={test_rmse:.4f}, MAE={test_mae:.4f}, MAPE={test_mape:.2f}%")
            
            all_rmse.append(test_rmse)
            all_mae.append(test_mae)
            all_mape.append(test_mape)

        # 최종 평균 계산 및 출력
        avg_rmse = np.mean(all_rmse)
        std_rmse = np.std(all_rmse)
        avg_mae = np.mean(all_mae)
        std_mae = np.std(all_mae)
        avg_mape = np.mean(all_mape)
        std_mape = np.std(all_mape)

        print("\n\n==================== 최종 5회 반복 평균 결과 ====================")
        print(f" - 평균 RMSE : {avg_rmse:.4f} +/- {std_rmse:.4f}")
        print(f" - 평균 MAE  : {avg_mae:.4f} +/- {std_mae:.4f}")

if __name__ == "__main__":
    main()

Using device: cuda
✅ JSON 파일 로드 성공.
SBERT 모델 로딩 중...
SBERT 모델 로딩 완료.
SBERT 문맥 벡터 생성 중...


Batches: 100%|██████████| 8681/8681 [03:02<00:00, 47.51it/s] 



--- 1단계: 하이퍼파라미터 랜덤 서치 (10회) 시작 ---

--- 시도 1/10 --- 파라미터: {'embed_dim': 64, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 128, 'patience': 7}
    --> 시도 1 검증 RMSE: 0.7879
    --> 새로운 최적 RMSE 발견: 0.7879 (파라미터: {'embed_dim': 64, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 128, 'patience': 7})

--- 시도 2/10 --- 파라미터: {'embed_dim': 128, 'hidden_dims': [256, 128], 'learning_rate': 0.001, 'batch_size': 256, 'patience': 7}
    --> 시도 2 검증 RMSE: 0.8274

--- 시도 3/10 --- 파라미터: {'embed_dim': 32, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 256, 'patience': 5}
    --> 시도 3 검증 RMSE: 0.7698
    --> 새로운 최적 RMSE 발견: 0.7698 (파라미터: {'embed_dim': 32, 'hidden_dims': [64, 32], 'learning_rate': 0.001, 'batch_size': 256, 'patience': 5})

--- 시도 4/10 --- 파라미터: {'embed_dim': 128, 'hidden_dims': [64, 32], 'learning_rate': 0.002, 'batch_size': 512, 'patience': 5}
    --> 시도 4 검증 RMSE: 0.8294

--- 시도 5/10 --- 파라미터: {'embed_dim': 128, 'hidden_dims': [128, 64],